In [2]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

from renetti.ws.spiders.utils import parse_product_json_ld_from_page

In [100]:
# url = "https://gymequipment.co.uk/exigo-chin-dip-leg-raise-station"
# url = "https://www.ukgymequipment.com/cardio-machines-c11/cross-trainers-c38/refurbished-772at-arc-trainer-p4131"

# # Techno Gym
url = "https://www.technogym.com/en-GB/product/sling-trainer_A0001317.html"
# url = "https://www.technogym.com/en-GB/product/power-bands_A0001235.html"
# url = "https://www.technogym.com/en-GB/product/dual-adjustable-pulley-fitness_MB44.html"
# url = "https://www.technogym.com/en-GB/product/excite-top_DF7U-SLS.html"
# url = "https://www.technogym.com/en-GB/product/technogym-bench_DGY100.html"

In [101]:
async with async_playwright() as playwright:
    async with await playwright.chromium.launch(headless=False) as browser:
        async with await browser.new_context() as context:
            async with await context.new_page() as page:
                await page.goto(url=url)
                html = await page.content()
                soup = BeautifulSoup(markup=html)
                scraped_equipment = parse_product_json_ld_from_page(soup=soup)

                glb_frame = None
                glb_image = None
                for frame in page.frames:
                    if "londondynamics.com" in frame.url:
                        glb_frame = frame
                        await glb_frame.wait_for_selector('model-viewer')
                        break

                if glb_frame:
                    frame_content = await glb_frame.content()
                    frame_soup = BeautifulSoup(markup=frame_content)
                    model_viewer = frame_soup.find("model-viewer")
                    if model_viewer:
                        glb_image = model_viewer.get("src")
                
                if glb_image:
                    scraped_equipment["image_links"].append(glb_image)

In [102]:
scraped_equipment

{'name': 'Sling Trainer',
 'image_links': ['https://www.technogym.com/en-GB/feed/images/A0001317/sling-trainer-plp.jpg'],
 'mpn': 'A0001317',
 'description': 'Improve muscle tone and stability with a functional workout of varying intensity that you can perform anywhere simply using your body weight.',
 'brands': ['Technogym'],
 'categories': [],
 'skus': ['A0001317']}